In [8]:
"""
Concatena datos de las estaciones CENTRO, SUROESTE y SUROESTE2
de los archivos.
Estandariza la dirección del viento a 'WD' y deja el orden de columnas como:
Fecha, Estacion, CO, NO, NO2, NOX, O3, PM10, PM2.5, PRS, RAINF, RH, SO2, SR, TOUT, WSR, WD, ...
"""

import pandas as pd
from pathlib import Path

FILE_2020_2021 = Path("Bases_Datos/Datos2020-2021.xlsx")
FILE_2022_2023 = Path("Bases_Datos/Datos2022-2023.xlsx")
FILE_2023_2024 = Path("Bases_Datos/Datos2023-2024.xlsx")

# Estaciones a incluir
STATIONS = ["CENTRO", "SUROESTE", "SUROESTE2"]

OUT_CSV  = Path("Bases_Datos/datos_anios.csv")

# Sinónimos a estandarizar -> WD
RENAME_EQUIV = {
    "WDV": "WD",
    "WDR": "WD",
    "WDR/WD": "WD",
    "WDR/WDV": "WD",
    "WIND_DIR": "WD",
}

PREFERRED_PARAM_ORDER = [
    "CO","NO","NO2","NOX","O3","PM10","PM2.5","PRS","RAINF","RH","SO2","SR","TOUT","WSR","WD"
]

def detect_date_col(df: pd.DataFrame):
    for c in df.columns:
        cl = str(c).strip().lower()
        if cl in ("date", "fecha", "time", "hora", "datetime"):
            return c
    for c in df.columns:
        if "date" in str(c).lower():
            return c
    return None

def unify_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Estandariza nombres equivalentes (WD variantes -> WD).
    No cambia otros nombres para respetar tu nomenclatura original.
    """
    rename_map = {c: RENAME_EQUIV[c] for c in df.columns if c in RENAME_EQUIV}
    return df.rename(columns=rename_map)

def order_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ordena columnas: Fecha, Estacion, luego parámetros en PREFERRED_PARAM_ORDER
    y al final cualquier columna restante (en orden alfabético para consistencia).
    """
    cols = list(df.columns)
    left = [c for c in ["Fecha", "Estacion"] if c in cols]
    preferred = [c for c in PREFERRED_PARAM_ORDER if c in cols]
    rest = [c for c in cols if c not in set(left + preferred)]
    rest_sorted = sorted(rest)  
    new_order = left + preferred + rest_sorted
    return df.reindex(columns=new_order)

def tidy_by_sheet(file_path: Path, stations):
    """
    Lee un archivo donde cada hoja es una estación (mismas columnas),
    renombra la columna de fecha a 'Fecha', agrega 'Estacion' y unifica WD.
    """
    out = []
    for st in stations:
        df = pd.read_excel(file_path, sheet_name=st)
        date_col = detect_date_col(df)
        if date_col is None:
            raise ValueError(f"No se encontró columna de fecha en {file_path.name}/{st}")
        df = df.rename(columns={date_col: "Fecha"})
        df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")
        df["Estacion"] = st
        df = unify_column_names(df)
        out.append(df)
    return pd.concat(out, ignore_index=True)

def tidy_param_horarios(file_path: Path, stations):
    """
    La hoja Param_horarios_Estaciones está en formato ancho:
      - Fila 0: nombre de estación repetido por bloque de variables (p. ej. 'CENTRO')
      - Fila 1: nombres de las variables (CO, NO, NO2, ...)
      - Fila 2: unidades (ppm, ppb, etc.) -> se ignoran
      - Columna 0: 'date' (fila 2 contiene el texto 'date'; los datos empiezan en la fila 3)
    Toma solo los bloques de las estaciones solicitadas y devuelve un DataFrame "largo".
    """
    raw = pd.read_excel(file_path, sheet_name="Param_horarios_Estaciones", header=None)

    data_start_row = 3  # después de 'date' (fila 2)
    date_series = pd.to_datetime(raw.iloc[data_start_row:, 0], errors="coerce")

    parts = []
    for st in stations:
        cols_idx = raw.columns[(raw.iloc[0, :] == st)].tolist()
        if not cols_idx:
            continue
        param_names = raw.iloc[1, cols_idx].tolist()
        df_st = raw.iloc[data_start_row:, cols_idx].copy()
        df_st.columns = param_names
        df_st.insert(0, "Fecha", date_series.values)
        df_st["Estacion"] = st
        df_st = unify_column_names(df_st)
        parts.append(df_st)

    if not parts:
        return pd.DataFrame(columns=["Fecha", "Estacion"])

    return pd.concat(parts, ignore_index=True)

def main():
    tidy_2020_2021 = tidy_by_sheet(FILE_2020_2021, STATIONS)
    tidy_2022_2023 = tidy_by_sheet(FILE_2022_2023, STATIONS)

    tidy_2023_2024 = tidy_param_horarios(FILE_2023_2024, STATIONS)

    # Alinear columnas por unión
    all_cols = sorted(set().union(
        *[df.columns for df in [tidy_2020_2021, tidy_2022_2023, tidy_2023_2024]]
    ))
    tidy_2020_2021 = tidy_2020_2021.reindex(columns=all_cols)
    tidy_2022_2023 = tidy_2022_2023.reindex(columns=all_cols)
    tidy_2023_2024 = tidy_2023_2024.reindex(columns=all_cols)

    # Concatenar
    df_all = pd.concat([tidy_2020_2021, tidy_2022_2023, tidy_2023_2024], ignore_index=True)

    # Limpieza: descartar filas sin fecha y ordenar
    df_all = df_all[~df_all["Fecha"].isna()].copy()
    df_all = df_all.sort_values(["Estacion", "Fecha"]).reset_index(drop=True)

    df_all = order_columns(df_all)

    # Guardar resultados
    OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    df_all.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

if __name__ == "__main__":
    main()

✅ Concatenación completa
 - Filas: 136,983
 - Columnas: 17
 - Estaciones: ['CENTRO', 'SUROESTE', 'SUROESTE2']
 - Rango de fechas: 2020-01-01 00:00:00  →  2024-07-31 23:00:00
 - CSV:  Bases_Datos/datos_anios.csv
